In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [2]:
df_original = pd.read_csv("./data/4_13_deliveries.csv")

df = df_original.drop(columns = ['First Name', 'Last Name', 'Notes',
                                'Phone #'])
df.head()

,ID,Street #,Street,Apartment
0,1,250,Kendall Street,NaN
1,2,364,Rindge Ave,17J
2,3,362,Rindge Ave,11M
3,4,364,Rindge Ave,21 c
4,5,30,Cambridge Park Drive,134


In [3]:
#data cleaning
df['Street_clean']=df['Street']
for address in df['Street_clean']:
    try:
        df.loc[df.Street_clean==address,'Street_clean']=address.replace(".","")
        df.loc[df.Street_clean==address,'Street_clean']=address.replace(",","")
        df.loc[df.Street_clean==address,'Street_clean']=address.replace(";","")
        df.loc[df.Street_clean==address,'Street_clean']=address.replace("/","")
        x=address.lower()
        
        if x[:2].isdigit():
            df.loc[df.Street_clean==address,'Street_clean']=x[x.find(" "):]

        if "-" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("-")]
        if "(" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("(")]
        if "calendar stret" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Callender Street"
        if "auburn" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Auburn Street"
        if "linwood pl" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Broadway"
        if "mass ave" in x or "":
            df.loc[df.Street_clean==address,'Street_clean']="Massachusetts Avenue"
        if "grant" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Grand Street"
        if  "aveue" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("aveue")] + "Avenue"
        if "stret" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("Stret")] + "Street"
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("stret")] + "Street"
        if  "cambridge park" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Cambridgepark Drive"
        if "lansdown" in x or "lansdowne" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Landsdowne St"
        if " apt " in x :
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("apt")]
        if "frankln" in x or 'franlin' in x or 'frankin' in x or "frtanklin" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Franklin Street"
        if "jackon" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Jackson Pl"
        if "eerie" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Erie Street"
        if "8th" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Eighth Street"
    except:
        pass

In [4]:
df['ADDRESS']=""
for address in df['ADDRESS']:
    if address == "":
        df['ADDRESS'] = df['Street #'].astype(str) + ' ' + \
                        df['Street_clean'] + ',' + \
                        "Cambridge" + ',' + \
                        "Massachusetts" + ',' + ' USA'   


In [5]:
good_points = pd.read_csv("point_table.csv")

In [6]:
len(good_points)

1565

In [10]:
smaller_df = good_points[['ADDRESS','location', 'point', 
                          'latitude', 'longitude','altitude', 
                          'point_location']].drop_duplicates()



In [11]:
point_table= df.merge(smaller_df,on = ['ADDRESS'],how='left')
bad_table = point_table.loc[point_table['point'].isnull()]
good_table = point_table.dropna(subset=['point'], how='all')


In [12]:
len(good_table), len(bad_table), len(df)

(331, 21, 352)

In [13]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
bad_table['location'] = bad_table['ADDRESS'].apply(geocode)
bad_table['point'] = bad_table['location'].apply(lambda loc: tuple(loc.point) if loc else None)


/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
bad_table[['latitude', 'longitude', 'altitude']] = pd.DataFrame(bad_table['point'].tolist(), index=bad_table.index)
bad_table['point_location'] = bad_table["longitude"].astype(str) + "," + bad_table["latitude"].astype(str)

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
null_table = bad_table.loc[bad_table['point'].isnull()]
null_table

,ID,Street #,Street,Apartment,Street_clean,ADDRESS,location,point,latitude,longitude,altitude,point_location
110,111,31,Newtone Ct,#135,Newtone Ct,"31 Newtone Ct,Cambridge,Massachusetts, USA",None,None,NaN,NaN,NaN,"nan,nan"
198,199,11,Newtown Ct,48,Newtown Ct,"11 Newtown Ct,Cambridge,Massachusetts, USA",None,None,NaN,NaN,NaN,"nan,nan"
295,296,3,Newtown Court,#11,Newtown Court,"3 Newtown Court,Cambridge,Massachusetts, USA",None,None,NaN,NaN,NaN,"nan,nan"


In [16]:
df_final = pd.concat([good_table,bad_table])

In [17]:
df_new = df_final.dropna(subset=['point'], how='all')
df_new.latitude.isnull().sum()


0

In [18]:
map1 = folium.Map(
    location=[42.362750, -71.101182],
    tiles='cartodbpositron',
    zoom_start=12,
)

df_new.apply(lambda row:folium.Marker(
    location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [19]:
locations = df_new[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [20]:
color_dict = {-1:'darkgreen',0:'darkpurple',1:'green',2:'purple',3:'red',4:'gray',5:'lightblue',
             6:'beige',7:'cadetblue',8:'black',9:'pink',10:'darkred',11:'lightred',
             12:'darkblue',13:'lightgreen',13:'white',14:'darkgreen',15:'darkpurple',16:'green',
              17:'purple',18:'red',19:'gray',20:'lightblue',
             21:'beige',22:'cadetblue',23:'black',24:'pink',25:'darkred',26:'lightred',
             27:'darkblue',28:'lightgreen',29:'white',30:'gray',31:'cadetblue',32:'orange',33:'green',
              34:'white',35:'darkgreen'}

In [21]:
neighborhood_list = ['Old Cambridge','Strawberry Hill','Inman Square',
                    'Cambridgeport','East Cambridge','Huron Village',
                    'North Cambridge','Porter Square','Waverley','Avon Hill',
                    'Union Square','University Park']

In [22]:
neighborhood_list_temp = []

for x in df_new['location']:
    z=0
    for y in neighborhood_list:
        if y in str(x):
            neighborhood_list_temp.append(y)
            z=1
            break
    if z==0:
        neighborhood_list_temp.append('Cambridge')

In [24]:
df_new['neighborhood']=neighborhood_list_temp
df_new['neighborhood'].value_counts()

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Cambridgeport      144
North Cambridge     48
East Cambridge      41
Inman Square        40
Cambridge           35
Avon Hill           10
Strawberry Hill      9
Porter Square        8
Huron Village        6
Old Cambridge        3
University Park      2
Waverley             2
Union Square         1
Name: neighborhood, dtype: int64

In [25]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(df_new['neighborhood'])
# Drop column B as it is now encoded
df_new = df_new.drop('neighborhood',axis = 1)
# Join the encoded df
df_new = df_new.join(one_hot)

In [27]:
df_temp = df_new.drop(columns = ['ID', 'Street #', 'Street', 'Apartment', 'Street_clean', 'ADDRESS',
       'location', 'point','altitude','point_location'])


In [28]:
from k_means_constrained import KMeansConstrained
# Convert DataFrame to matrix
mat = df_temp.values
# Using sklearn
km = KMeansConstrained(
     n_clusters=35,
     size_min=9,
     size_max=13,
     random_state=0
).fit(mat)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
df_new['kmeans_cluster'] = labels
df_new['kmeans_cluster'].value_counts()

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


23    13
14    13
16    13
3     13
22    13
24    12
20    12
34    12
26    10
25    10
21    10
4     10
18    10
6      9
11     9
5      9
7      9
8      9
2      9
9      9
1      9
10     9
17     9
12     9
13     9
15     9
33     9
19     9
27     9
28     9
29     9
30     9
31     9
32     9
0      9
Name: kmeans_cluster, dtype: int64

In [29]:
df_new['kmeans_color']=df_new['kmeans_cluster'].map(color_dict)

In [30]:
df_new = df_new.reset_index()


In [31]:
map = folium.Map(location=[42.379750, -71.101182], zoom_start=13)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=('Driver Group: '+str(df_new['kmeans_cluster'][point])),
                 icon=folium.Icon(color=df_new["kmeans_color"][point])).add_to(map)
    
map.save('DeliveryMaps_4_13.html')
map

In [32]:
df_new['Driver_ID']=df_new['kmeans_cluster']
null_table['Driver_ID']=99

df_new = df_new.drop(columns = ['index','altitude',
       'point_location', 'Avon Hill', 'Cambridge', 'Cambridgeport', 'East Cambridge',
       'Huron Village', 'Inman Square', 'North Cambridge', 'Old Cambridge',
       'Porter Square', 'Strawberry Hill', 'Union Square', 'University Park',
       'Waverley','kmeans_cluster','kmeans_color'])


/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
driver_table = pd.concat([null_table,df_new])

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [34]:
driver_table.head()

,ADDRESS,Apartment,Driver_ID,ID,Street,Street #,Street_clean,altitude,latitude,location,longitude,point,point_location
110,"31 Newtone Ct,Cambridge,Massachusetts, USA",#135,99,111,Newtone Ct,31,Newtone Ct,NaN,NaN,None,NaN,None,"nan,nan"
198,"11 Newtown Ct,Cambridge,Massachusetts, USA",48,99,199,Newtown Ct,11,Newtown Ct,NaN,NaN,None,NaN,None,"nan,nan"
295,"3 Newtown Court,Cambridge,Massachusetts, USA",#11,99,296,Newtown Court,3,Newtown Court,NaN,NaN,None,NaN,None,"nan,nan"
0,"364 Rindge Ave,Cambridge,Massachusetts, USA",17J,22,2,Rindge Ave,364,Rindge Ave,NaN,42.393079,"364, Rindge Avenue, North Cambridge, Cambridge...",-71.138877,"(42.39307855, -71.13887697600367, 0.0)",NaN
1,"362 Rindge Ave,Cambridge,Massachusetts, USA",11M,3,3,Rindge Ave,362,Rindge Ave,NaN,42.392533,"362, Rindge Avenue, North Cambridge, Cambridge...",-71.138152,"(42.3925332, -71.13815172576868, 0.0)",NaN


In [35]:
driver_table_final = driver_table.merge(df_original[['ID','First Name', 'Last Name', 'Notes',
                                'Phone #']], on = 'ID')

In [36]:
df = df_original.drop(columns = ['First Name', 'Last Name', 'Notes',
                                'Phone #'])


In [37]:
driver_table_final.columns

Index(['ADDRESS', 'Apartment', 'Driver_ID', 'ID', 'Street', 'Street #',
       'Street_clean', 'altitude', 'latitude', 'location', 'longitude',
       'point', 'point_location', 'First Name', 'Last Name', 'Notes',
       'Phone #'],
      dtype='object')

In [38]:
driver_table_final=driver_table_final.drop(columns = 
                                          ['altitude',
                                          'longitude',
                                          'latitude',
                                          'location',
                                          'point',
                                          'point_location',
                                          'Street_clean'])


In [39]:
driver_table_final=driver_table_final.sort_values(['Driver_ID'])

In [42]:
driver_table_final.to_excel("Driver_Table_4_13.xlsx", index=False)